In [1]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

In [2]:
x_len = 1
y_len = 1
nx    = 10
ny    = 10
edge_size_x = 1/nx
edge_size_y = 1/ny

# Create mesh and Functional space
#mesh  = UnitSquareMesh (nx,ny)
mesh   = RectangleMesh(Point(0, 0), Point(x_len, y_len), nx, ny)

Qe      = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
Be      = FiniteElement("Bubble",   mesh.ufl_cell(), 4)
Ve      = VectorElement(NodalEnrichedElement(Qe, Be))
element = MixedElement(Ve,Qe)
W       = FunctionSpace(mesh, element)

# Define trial and test functions
w       = Function(W)
(u, p)  = TrialFunctions(W)
(v, q)  = TestFunctions(W)
n       = FacetNormal(mesh)

sub_domains = MeshFunction('size_t', mesh, 1)
ds = Measure('ds', domain=mesh, subdomain_data=sub_domains)

# Construct a spatially-varying permeability matrix (inverse)
k   = "1.0"
k11 = Expression(k, degree = 2)
k12 = Constant(0.0)
k21 = Constant(0.0)
k22 = Expression(k, degree = 2)
K   = as_matrix(((k11, k12), (k21, k22)))

# Costruct bilinear and linear parts of equation
a   = (dot(K*u,v) - div(v)*p - div(u)*q)*dx
L   = Constant(1)*dot(n,v)*ds     #(1)  + Constant(2)*dot(n,v)*ds(2)# f*dot(n,v)*ds

# Create arrays of coordinates and zero_array of fill_factor

#coor   = mesh.coordinates()         # np.array (121, 2); all coordinates
# array of facet midpoints 
x_a = np.arange(0, x_len + edge_size_x, edge_size_x)
y_a = np.arange(0, y_len + edge_size_y, edge_size_y)

#array([0.   , 0.075, 0.15 , 0.25 , 0.35 , 0.45 , 0.55 , 0.65 , 0.75 , 0.85 , 0.95 , 1.   ]) include boundart dots
vertical_facet_centers_x        = np.copy(x_a)[:-1] + edge_size_x / 2 
vertical_facet_centers_x        = np.insert(vertical_facet_centers_x, 0, 0)
vertical_facet_centers_x        = np.append(vertical_facet_centers_x, x_len)

#array([0.025, 0.1  , 0.2  , 0.3  , 0.4  , 0.5  , 0.6  , 0.7  , 0.8  , 0.9  , 0.975])
vertical_facet_centers_y        = np.copy(y_a)
vertical_facet_centers_y[0]    += edge_size_y / 4
vertical_facet_centers_y[-1]   -= edge_size_y / 4

#array([0.  , 0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.  ]) include boundart dots
horizontal_facet_centers_y      = np.copy(y_a)[:-1] + edge_size_y / 2
horizontal_facet_centers_y      = np.insert(horizontal_facet_centers_y, 0, 0)
horizontal_facet_centers_y      = np.append(horizontal_facet_centers_y, y_len)

#array([0.025, 0.1  , 0.2  , 0.3  , 0.4  , 0.5  , 0.6  , 0.7  , 0.8  , 0.9  , 0.975])
horizontal_facet_centers_x      = np.copy(x_a)
horizontal_facet_centers_x[0]  += edge_size_x / 4
horizontal_facet_centers_x[-1] -= edge_size_x / 4

# np.array(121, 2), where cell_center[i,0] - x coord, cell_center[i,1] - y coord 
xx, yy = np.meshgrid(horizontal_facet_centers_x, vertical_facet_centers_y)
cell_centers = np.array((xx.flatten(), yy.flatten())).T
    
# array of facet length
# array([0.05, 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.05])
length_hor      = np.zeros(nx+1)
length_hor.fill(edge_size_x)
length_hor[0]  /= 2
length_hor[-1] /= 2

length_ver      = np.zeros(ny+1)
length_ver.fill(edge_size_y)
length_ver[0]  /= 2
length_ver[-1] /= 2

#fill factor array
fill_factor      = np.zeros ((nx+1,ny+1))
fill_factor[1,0] = 1
ff_array         = fill_factor.flatten()

tol = 1E-15

def U (fill_factor):
    def b_inj(x):
            return (abs(x[0] - 0.0) < tol) and (abs(x[1] - 0.1)  < tol)         # Injection coordinates  (1,1)
        
    def b_out(x):
            x    = x.reshape((1, 2))
            dist = np.linalg.norm(x - cell_centers, axis=1)
            x_ff = fill_factor.ravel()[np.argmin(dist)]
            return x_ff < 1
    
    def walls(x, on_boundary):
            return near(x[0],0) or near(x[0], x_len) or near(x[1],0) or near(x[1], y_len) and on_boundary 
    # Create BC
    bc_n     = DirichletBC(W.sub(0), Constant ((0,0)), walls)                        # Neumann on the walls 
    bc_in    = DirichletBC(W.sub(1), Constant (1),     b_inj,  method='pointwise')   # Dirichlet (p=1) on injection point (0,0.1)
    bc_out   = DirichletBC(W.sub(1), Constant (0),     b_out,  method='pointwise')

    # Solving equation and plot results
    solve(a == L, w, bcs = [bc_n, bc_in, bc_out])
    (u, p)   = w.split()
    
    #flow in control volume from all (4) facets of control volume
    def Q (i,j):
        vertical_facets   = length_ver[i] * (u(vertical_facet_centers_x[i],   vertical_facet_centers_y[j])[0]     \
                                           - u(vertical_facet_centers_x[i+1], vertical_facet_centers_y[j])[0])
        
        horizontal_facets = length_hor[j] * (u(horizontal_facet_centers_x[i], horizontal_facet_centers_y[j])[1] \
                                           - u(horizontal_facet_centers_x[i], horizontal_facet_centers_y[j+1])[1])
        sum = vertical_facets + horizontal_facets
        sum = max(0,sum)
        if fill_factor[i,j]==1:
            sum=0
        return sum

    # Array of flow values in [i,j] control volume        
    Q_arr = np.zeros ((ny+1,nx+1))                  
    for i in range (ny+1):
        for j in range (nx+1):
            Q_arr[i,j] = Q(i,j)
            
    Q_arr = np.where (Q_arr<0, 0, Q_arr)
    dt = 1/Q_arr.max()
    
    fill_factor  += Q_arr*dt
    fill_factor   = np.clip(fill_factor, 0, 1)

    plot(u, title = 'Velocity', mesh=mesh)
    plt.show()
    plot(p, title = 'Pressure', mesh=mesh)
    plt.show()
    file = File("pressure.pvd")
    file << w.split()[1]

    # array of midpoints 

    fig = go.Figure(data=go.Heatmap(x = xx.flatten(), y = yy.flatten(), z = fill_factor.flatten(), type = 'heatmap',colorscale = 'Viridis'))
    fig.update_layout(width = 400, height = 400, autosize = False, title = 'Fill Factor' )
    fig.show()
    
    return fill_factor

In [3]:
while not np.allclose(fill_factor, 1):
    fill_factor = U(fill_factor)
    ff_array = np.vstack((ff_array, fill_factor.flatten()))

In [4]:
heatmaps = [ff_array[i] for i in range(ff_array.shape[0])]
fig = go.Figure(data=go.Heatmap(x=xx.flatten(), y=yy.flatten(), z=heatmaps), 
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(ff_array.shape[0])])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True, 
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.update_layout(width = 400, height = 400, autosize = False, title = 'Fill Factor',)
fig.show()